# import libraries and package


In [ ]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from torch.utils.data import Dataset, Subset, DataLoader, ConcatDataset

from torchvision import transforms
from torchvision.models import resnet34

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from PIL import Image
from copy import deepcopy

righe di codice utili per importare da altri jupyter notebook delle funzioni. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
%cd "/content/drive/MyDrive/MLDL2021_Calderaro_Giannuzzi_Scorca/File colab/File progetto"
!pip install import-ipynb

import import_ipynb                           

In [ ]:
#retrieve from folder 'data'
%cd "/content/drive/MyDrive/MLDL2021_Calderaro_Giannuzzi_Scorca/File colab/File progetto/data"                          
from DataClean import Cifar100 as datasetManager   

#retrieve from folder 'models'
%cd "/content/drive/MyDrive/MLDL2021_Calderaro_Giannuzzi_Scorca/File colab/File progetto/models"
from resnet import resnet32
from Manager import Manager

#retrieve from folder 'extension'
%cd "/content/drive/MyDrive/MLDL2021_Calderaro_Giannuzzi_Scorca/File colab/File progetto/owr"
from iCarlwithRejectFC import iCaRLWithRejectFC

#retrieve from folder 'logs'
%cd "/content/drive/MyDrive/MLDL2021_Calderaro_Giannuzzi_Scorca/File colab/File progetto/logs"
from Save_logs import save_logs

# return in the main project folder
%cd "/content/drive/MyDrive/MLDL2021_Calderaro_Giannuzzi_Scorca/File colab/File progetto"

# Arguments

In [ ]:
DEVICE=torch.device("cuda:0")
DIR='/content/data'

RANDOM_SEED = [1993, 30, 423]     # choose and put here one seed for each different run

NUM_CLASSES = 100       # Total number of classes

# Training
BATCH_SIZE = 64         # Batch size 
NUM_EPOCHS = 70         # Total number of training epochs
LR = 2                  # Initial learning rate
MOMENTUM = 0.9          # Momentum for stochastic gradient descent (SGD)
WEIGHT_DECAY = 1e-5     # Weight decay from iCaRL
MILESTONES = [49, 63]   # Step down policy from iCaRL (MultiStepLR)
                        # Decrease the learning rate by gamma at each milestone
GAMMA = 0.2             # Gamma factor from iCaRL

In [ ]:
train_transform = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                    
])



# Closed World With Rejeciton and Open World With Rejection


In [ ]:
from iCarlwithRejectFC import iCaRLWithRejectFC

logs_iCaRLwithReject = [[] for _ in range(len(RANDOM_SEED))]

CUDA_LAUNCH_BLOCKING=1
NUM_RUNS = len(RANDOM_SEED)
NUM_EPOCHS = 70

threshold = 0.80
RUN_NAME_CLOSED = 'icarl_rejectFC_newClosed_threshold80'
RUN_NAME_OPEN = 'icarl_rejectFC_newOpen_threshold80'



# Initialize logs
logs_iCaRLwithReject_closed = [[] for _ in range(NUM_RUNS)]
logs_iCaRLwithReject_open = [[] for _ in range(NUM_RUNS)]


for run_i in range(NUM_RUNS):

  train_dataset = datasetManager(DIR, train=True, download=True, random_state=RANDOM_SEED[run_i], transform=train_transform)
  test_dataset_open = datasetManager(DIR, train=False, download=False, random_state=RANDOM_SEED[run_i], transform=test_transform)
  test_dataset_closed = datasetManager(DIR, train=False, download=False, random_state=RANDOM_SEED[run_i], transform=test_transform)


    
  net = resnet32()
  icarl = iCaRLWithRejectFC(DEVICE, net, LR, MOMENTUM, WEIGHT_DECAY, MILESTONES, GAMMA, NUM_EPOCHS, BATCH_SIZE, train_transform, test_transform)

  test_dataset_open.set_classes_batch([test_dataset_open.batch_splits[i] for i in range(5, 10)])


  for split_i in range(5):
        print(f"## Split {split_i} of run {run_i} ##")


        train_dataset.set_classes_batch(train_dataset.batch_splits[split_i])
        test_dataset_closed.set_classes_batch([test_dataset_closed.batch_splits[i] for i in range(0, split_i+1)])
        

        train_idx, val_idx = train_test_split(list(range(len(train_dataset))),      
                                          random_state=RANDOM_SEED[run_i],
                                          test_size = 0.1)   
                                          
        train_data_split = Subset(train_dataset , train_idx)                       
        val_data_split = Subset(train_dataset , val_idx)                           
        
        train_logs = icarl.incremental_train(split_i, train_data_split, val_data_split)
        


            #closed world
        logs_iCaRLwithReject_closed[run_i].append({})

        acc, unk, all_targets, all_preds = icarl.test_without_classifier(test_dataset_closed, threshold)

        logs_iCaRLwithReject_closed[run_i][split_i]['accuracy'] = acc
        logs_iCaRLwithReject_closed[run_i][split_i]['unknown'] = unk
        logs_iCaRLwithReject_closed[run_i][split_i]['conf_mat'] = confusion_matrix(all_targets.to('cpu'), all_preds.to('cpu'))

    


            #open world
        logs_iCaRLwithReject_open[run_i].append({})

        acc, unk, all_targets, all_preds = icarl.test_without_classifier(test_dataset_open, threshold)

        logs_iCaRLwithReject_open[run_i][split_i]['accuracy'] = acc
        logs_iCaRLwithReject_open[run_i][split_i]['unknown'] = unk
        logs_iCaRLwithReject_open[run_i][split_i]['conf_mat'] = confusion_matrix(all_targets.to('cpu'), all_preds.to('cpu'))

          

save_logs(logs_iCaRLwithReject_closed,RANDOM_SEED, NUM_EPOCHS, RUN_NAME_CLOSED, BATCH_SIZE)
save_logs(logs_iCaRLwithReject_open,RANDOM_SEED, NUM_EPOCHS, RUN_NAME_OPEN, BATCH_SIZE)